# Modeling Part 2. Multivariate Time Series Analysis.

## Importing Necessary Libraries & Loading Data

In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm


from datetime import datetime

import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import pmdarima as pm

# This will allow us to avoid a FutureWarning when plotting.
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [7]:
data = pd.read_csv('../Data/Data_concat.csv')
data.head()

,DATE,IA_PRCP,IL_PRCP,MN_PRCP,TMAX_IA,TMAX_IL,TMAX_MN,TMIN_IA,TMIN_IL,TMIN_MN,...,MN_TMIN_lag_1,IA_TMIN_lag_360,IL_TMIN_lag_360,MN_TMIN_lag_360,IA_TMAX_lag_1,IL_TMAX_lag_1,MN_TMAX_lag_1,IA_TMAX_lag_360,IL_TMAX_lag_360,MN_TMAX_lag_360
0,1990-01-02,0.00,0.00,0.0,49.0,45.0,27.0,28.0,25.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1990-01-03,0.00,0.00,0.0,45.0,49.0,32.0,35.0,32.0,19.0,...,1.0,NaN,NaN,NaN,49.0,45.0,27.0,NaN,NaN,NaN
2,1990-01-04,0.94,0.05,0.0,42.0,47.0,26.0,21.0,32.0,1.0,...,19.0,NaN,NaN,NaN,45.0,49.0,32.0,NaN,NaN,NaN
3,1990-01-05,0.00,0.00,0.0,40.0,38.0,20.0,18.0,16.0,11.0,...,1.0,NaN,NaN,NaN,42.0,47.0,26.0,NaN,NaN,NaN
4,1990-01-08,0.00,0.00,0.0,43.0,50.0,40.0,31.0,21.0,16.0,...,11.0,NaN,NaN,NaN,40.0,38.0,20.0,NaN,NaN,NaN


In [8]:
data['DATE'] = pd.to_datetime(data['DATE'])
data.set_index('DATE', inplace=True)
data.sort_index(inplace=True)
data

,IA_PRCP,IL_PRCP,MN_PRCP,TMAX_IA,TMAX_IL,TMAX_MN,TMIN_IA,TMIN_IL,TMIN_MN,Settlement Price,...,MN_TMIN_lag_1,IA_TMIN_lag_360,IL_TMIN_lag_360,MN_TMIN_lag_360,IA_TMAX_lag_1,IL_TMAX_lag_1,MN_TMAX_lag_1,IA_TMAX_lag_360,IL_TMAX_lag_360,MN_TMAX_lag_360
DATE,,,,,,,,,,,,,,,,,,,,,
1990-01-02,0.00,0.00,0.00,49.0,45.0,27.0,28.0,25.0,1.0,5.613,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,0.00,0.00,0.00,45.0,49.0,32.0,35.0,32.0,19.0,5.673,...,1.0,NaN,NaN,NaN,49.0,45.0,27.0,NaN,NaN,NaN
1990-01-04,0.94,0.05,0.00,42.0,47.0,26.0,21.0,32.0,1.0,5.633,...,19.0,NaN,NaN,NaN,45.0,49.0,32.0,NaN,NaN,NaN
1990-01-05,0.00,0.00,0.00,40.0,38.0,20.0,18.0,16.0,11.0,5.645,...,1.0,NaN,NaN,NaN,42.0,47.0,26.0,NaN,NaN,NaN
1990-01-08,0.00,0.00,0.00,43.0,50.0,40.0,31.0,21.0,16.0,5.707,...,11.0,NaN,NaN,NaN,40.0,38.0,20.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-24,0.00,0.00,0.00,53.0,57.0,32.0,27.0,26.0,22.0,9.365,...,19.0,67.0,67.0,62.0,45.0,56.0,35.0,86.0,96.0,68.0
2019-12-26,0.00,0.01,0.01,62.0,68.0,33.0,32.0,46.0,22.0,9.465,...,22.0,62.0,68.0,63.0,53.0,57.0,32.0,78.0,86.0,80.0
2019-12-27,0.00,0.00,0.02,37.0,48.0,25.0,32.0,39.0,20.0,9.415,...,22.0,65.0,60.0,58.0,62.0,68.0,33.0,84.0,89.0,79.0


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7559 entries, 1990-01-02 to 2019-12-31
Data columns (total 25 columns):
IA_PRCP             7559 non-null float64
IL_PRCP             7559 non-null float64
MN_PRCP             7559 non-null float64
TMAX_IA             7559 non-null float64
TMAX_IL             7559 non-null float64
TMAX_MN             7559 non-null float64
TMIN_IA             7559 non-null float64
TMIN_IL             7559 non-null float64
TMIN_MN             7559 non-null float64
Settlement Price    7559 non-null float64
IA_prcp_lag_1       7558 non-null float64
IL_prcp_lag_1       7558 non-null float64
MN_prcp_lag_1       7558 non-null float64
IA_TMIN_lag_1       7558 non-null float64
IL_TMIN_lag_1       7558 non-null float64
MN_TMIN_lag_1       7558 non-null float64
IA_TMIN_lag_360     7199 non-null float64
IL_TMIN_lag_360     7199 non-null float64
MN_TMIN_lag_360     7199 non-null float64
IA_TMAX_lag_1       7558 non-null float64
IL_TMAX_lag_1       7558 non-null f

## Modeling



In [5]:
train_ex = data[data.index<'2016-01-01'].drop(columns = ['IA_PRCP', 'IL_PRCP', 'MN_PRCP', 'TMAX_IA', 'TMAX_IL', 'TMAX_MN',
       'TMIN_IA', 'TMIN_IL', 'TMIN_MN']).dropna()
test_ex = data[data.index>='2016-01-01'].drop(columns = ['IA_PRCP', 'IL_PRCP', 'MN_PRCP', 'TMAX_IA', 'TMAX_IL', 'TMAX_MN',
       'TMIN_IA', 'TMIN_IL', 'TMIN_MN']).dropna()

In [6]:
sarimax = SARIMAX(endog = train_ex['Settlement Price'],
                order = (2,1,2),
                 seasonal_order = (1,0,0,365),
                 exog = train_ex.drop(columns=['Settlement Price']))

#fit SARIMAX model
model = sarimax.fit()

/Users/andreayoss/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/Users/andreayoss/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


KeyboardInterrupt: 

In [ ]:
preds = model.forecast(len(test_ex), 
               exog = test_ex.drop(columns = ['Settlement Price']),
               step=1, 
               alpha = 0.05)
        
#evaluate predictions
print(mean_absolute_error(test_ex['Settlement Price'], preds))

In [ ]:
#plot data
plt.figure(figsize = (10,6))
plt.plot(train_ex['Settlement Price'], color = 'blue')
plt.plot(test_ex['Settlement Price'], color = 'orange')
plt.plot(preds, color = 'green')
plt.title(label = 'Av Daily Soybean Price with SARIMAX(2,1,2)X(0,0,0,365)')
plt.show();

In [ ]:
# Convert training/ testing sets to a DataFrame.
X_train = pd.DataFrame(train_ex)
X_test = pd.DataFrame(test_ex)

# Add column for lagged values of precipitation.
X_train['precip_lag_1'] = X_train.shift(1)
X_test.shift(1)

#Check out top of the test dataframe
X_test[['precip_lag_1']].head()